In [1]:
#### filter conditions:
#### 1. only limit IP total number 
####    1.1 input (refCount, altCount >=2 and refCount+altCount >=10)
####    1.2 ip (refCount+altCount >=10)
#### 2. IP/Input >=2
#### 3. within m6A peak (remove m6Am)
#### filter order
#### remove SNPs which don't meet minimum needs
#### Hypothesis test
#### process 2,3

In [1]:
from scipy import stats
from multiprocessing import Pool
from statsmodels.stats.multitest import fdrcorrection

#############################################################
os.chdir("/Charles/project/ASm6A/by_ind/count/")
result_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/"
os.system("mkdir -p %s" % result_dir)
#############################################################


def generate_totalNum_dict():
    df_map = pd.read_table(
        "/home/galaxy/project/alleleSpecific_analysis/data/sample_info/total_Input_vs_IP.txt"
    )
    map_dict = dict(zip(df_map['IP'], df_map['Input']))
    summary_file = "/Charles/project/ASm6A/bam_count/AllSamples.bam_stat.txt"
    df_stat = pd.read_table(summary_file, header=None)
    number_dict = dict(
        zip(df_stat.iloc[:, 0].tolist(), df_stat.iloc[:, 1].tolist()))
    totalNum_dict = {}
    for sample in number_dict.keys():
        if sample in map_dict:
            totalNum_dict[sample] = [
                number_dict[sample], map_dict[sample],
                number_dict[map_dict[sample]]
            ]
    return totalNum_dict


def preprocess_df(ip_prefix):
    input_prefix = totalNum_dict[ip_prefix][1]
    ip_file, input_file = "ip_count/%s.readcounts.txt" % ip_prefix, "input_count/%s.readcounts.txt" % input_prefix
    df_ip, df_input = pd.read_table(ip_file), pd.read_table(input_file)
    df_results = []
    totalNum_list = [totalNum_dict[ip_prefix][0], totalNum_dict[ip_prefix][2]]
    treat_list, df_list = ['ip', 'input'], [df_ip, df_input]
    for i in range(len(df_list)):
        df, treat, total_num = df_list[i], treat_list[i], totalNum_list[i]
        df = df[[
            'contig', 'position', 'refAllele', 'altAllele', 'refCount',
            'altCount'
        ]]
        #######################################################
        if i == 0:  # ip sample: only restrict total number
            df = df[(df['refCount'] + df['altCount']) >= 10]
        else:
            df = df[(df['refCount'] >= 2) & (df['altCount'] >= 2)]
            df = df[(df['refCount'] + df['altCount']) >= 10]
        #####################################################################
        df['refRPKM_%s' % treat] = (df['refCount'] / total_num) * 1000000000
        df['altRPKM_%s' % treat] = (df['altCount'] / total_num) * 1000000000
        df_results.append(df)
    return df_results[0], df_results[1]


def fisher_exact_test_each(ip_prefix):
    result_file = os.path.join(result_dir, ip_prefix + ".txt")
    df_ip, df_input = preprocess_df(ip_prefix)
    df = df_ip.merge(df_input,
                     on=['contig', 'position', 'refAllele', 'altAllele'],
                     how='left').dropna(how="any")
    print(ip_prefix, len(df))
    df['refRPKM_ratio'] = df['refRPKM_ip'] / df['refRPKM_input']
    df['altRPKM_ratio'] = df['altRPKM_ip'] / df['altRPKM_input']
    df['allelicRatio'] = df['refRPKM_ratio'] / (df['altRPKM_ratio'] +
                                                df['refRPKM_ratio'])
    pvalue_list, odds_list = [], []
    for i, j in df.iterrows():
        a, b = j['refRPKM_ip'] + 0.5, j['altRPKM_ip'] + 0.5
        c, d = j['refRPKM_input'] + 0.5, j['altRPKM_input'] + 0.5
        oddsratio, pvalue = stats.fisher_exact([[a, b], [c, d]])
        pvalue_list.append(pvalue)
        odds_list.append(oddsratio)
    qvalue_list = list(fdrcorrection(pvalue_list, method="indep")[1])  # fdr_bh
    df['pvalue'], df['oddsratio'], df[
        'qvalue'] = pvalue_list, odds_list, qvalue_list
    df_sub = df[[
        'contig', 'position', 'refAllele', 'altAllele', 'refRPKM_ratio',
        'altRPKM_ratio', 'allelicRatio', 'pvalue', 'oddsratio', 'qvalue'
    ]]
    df_sub.to_csv(result_file, sep="\t", index=False)


totalNum_dict = generate_totalNum_dict()
prefix_list = totalNum_dict.keys()
# print(prefix_list)
pool = Pool(processes=30)
for ip_prefix in prefix_list:
    #     fisher_exact_test_each(ip_prefix)
    pool.apply_async(fisher_exact_test_each, (ip_prefix, ))
pool.close()
pool.join()

CRR055553 736
CRR073016 595
CRR073020 298
CRR042292 1340
CRR042294 1112
SRR8209849 2345
CRR073018 869
SRR8209863 1135
CRR042282 1543
SRR8209857 1429
SRR8209875 2119
SRR8209837 3486
SRR8209853 1794
SRR8209855 2896
SRR8209867 1274
CRR042284 3797
SRR8209859 985
CRR055527 3119
SRR8209865 1407
liver_IP_1 1789
SRR8209871 5459
SRR8209841 12936
muscle_IP_5 5675
heart_IP_2 3898
liver_IP_3 3094
muscle_IP_4 3730
lung_IP_4 4979
SRR8209839 16775
brain_IP_2 5761
brain_IP_3 6592
SRR8209843 754
SRR8209877 770
lung_IP_5 2430
SRR8209873 890
SRR8209847 7463
kidney_IP_4 5619
SRR8209879 2747
CRR055533 353
SRR8209861 4615
brain_IP_1 6826
CRR042286 1446
CRR042290 1259
heart_IP_3 3638
kidney_IP_2 7777
SRR8209869 3972
SRR8209845 11176
heart_IP_1 2240
liver_IP_2 3381
kidney_IP_3 4923
CRR042296 1306
CRR042310 957
CRR042320 497
placenta_IP_6 4106
CRR055555 390
CRR042300 1033
stomach_IP_4 3308
CRR042280 1744
CRR055539 393
CRR042316 1635
CRR042302 670
SRR8209881 6233
CRR055543 847
CRR055547 760
stomach_IP_5 2401
CR

#### continue to filter
##### 1. sig (qvalue < 0.05)
#####    1.1 within m6A peak
#####           1.1.1 ip/input >= 2 

######
##### 2. unsig (qvalue < 0.05)
#####     2.1 within m6A peak

In [3]:
def select_sig_and_unsig(in_file, data_type): # data_type: sig or unsig    
    df = pd.read_table(in_file)
    #############################
    if data_type == "sig":
        df = df[(df['qvalue']<0.05)] ##### 不同的过滤条件，用于不同的分析内容
        # df = df[(df['qvalue']<0.1)]
#         df = df[(df['pvalue']<0.05)]
    elif data_type == "unsig":
        df = df[(df['qvalue']>=0.05)] ####
        # df = df[(df['qvalue']>=0.1)]
#         df = df[(df['pvalue']>=0.05)]
    #############################
    df = df.round(2)
    df['mark'] = np.where(df['altRPKM_ratio'] > df['refRPKM_ratio'], "alt", "ref")
    df['start'] = df['position'] - 1
    df['term'] = df['refAllele'] + ">" + df['altAllele'] + ";" + df['allelicRatio'].astype(str) + ";" + df['mark']
    df = df[['contig', 'start', 'position', 'term', 'refRPKM_ratio', 'altRPKM_ratio']]
    return df
    
os.chdir("/Charles/project/ASm6A/Hypothesis_test/ASm6A/")
sig_result_dir, unsig_result_dir = "sig/", "unsig/"
# sig_result_dir, unsig_result_dir = "sigpv005/", "unsigpv005/" 
# sig_result_dir, unsig_result_dir = "sig01/", "unsig01/" ##########
os.system("mkdir -p %s" % sig_result_dir)
os.system("mkdir -p %s" % unsig_result_dir)
file_list = glob.glob("*.txt")
for x in file_list:
    df_sig, df_unsig = select_sig_and_unsig(x, "sig"), select_sig_and_unsig(x, "unsig")
    print(os.path.basename(x), len(df_sig), len(df_unsig))
    res_sig = os.path.join(sig_result_dir, x.replace(".txt",".bed"))
    res_unsig = os.path.join(unsig_result_dir, x.replace(".txt",".bed"))
    df_sig.to_csv(res_sig, sep="\t", header=False, index=False)
    df_unsig.to_csv(res_unsig, sep="\t", header=False, index=False)
    
    
def overlap_with_m6A(in_dir):
    os.chdir(in_dir)
    result_dir = "contained_m6A/"
    os.system("mkdir -p %s" % result_dir)
#     m6a_dir = "/Charles/project/ASm6A/peak_calling/union_three_bedtools/rm_m6Am/"
#     m6a_dir = "/Charles/project/ASm6A/peak_calling/union_peak_bedtools/rm_m6Am/"
    m6a_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_MSPC/rm_m6Am/" # kidney_IP_3_m6A.bed
    # m6a_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_bedtools/rm_m6Am/"
    # m6a_dir = "/Charles/project/ASm6A/peak_calling/macs2/10e5/rm_m6Am/"
    ##
    bed_list = glob.glob("%s/*.bed" % in_dir)
    for asm6a in bed_list:
        prefix = os.path.basename(asm6a).split(".")[0]
        m6a = os.path.join(m6a_dir, "%s_m6A.bed" % prefix)
        res = os.path.join(result_dir, os.path.basename(asm6a))
        ###################################################
        os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" % (asm6a, m6a, res))
        ####################################################################################
        
        
overlap_with_m6A("/Charles/project/ASm6A/Hypothesis_test/ASm6A/sig/")
overlap_with_m6A("/Charles/project/ASm6A/Hypothesis_test/ASm6A/unsig/")
# overlap_with_m6A("/Charles/project/ASm6A/Hypothesis_test/ASm6A/sigpv005/")
# overlap_with_m6A("/Charles/project/ASm6A/Hypothesis_test/ASm6A/unsigpv005/")


def select_highFC(in_dir):
    os.chdir(in_dir)
    result_dir = "highFC/"
    os.system("mkdir -p %s" % result_dir)
    bed_list = glob.glob("%s/*.bed" % in_dir)
    for asm6a in bed_list:
        df = pd.read_table(asm6a, header=None)
        df.columns = ['contig', 'start', 'position', 'term', 'refRPKM_ratio', 'altRPKM_ratio']
        df['mark'] = df['term'].str.split(";").str[2]
        #####################################################
        df = df[((df['mark']=="ref")&(df['refRPKM_ratio']>=2)) | ((df['mark']=="alt")&(df['altRPKM_ratio']>=2))]
        ####################################################
        res = os.path.join(result_dir, os.path.basename(asm6a))
        df.to_csv(res, sep="\t", header=False, index=False)
        
select_highFC("/Charles/project/ASm6A/Hypothesis_test/ASm6A/sig/contained_m6A/")
# select_highFC("/Charles/project/ASm6A/Hypothesis_test/ASm6A/sigpv005/contained_m6A/")

CRR055533.txt 333 20
SRR8209881.txt 4878 1355
CRR042298.txt 562 80
CRR042290.txt 1071 188
CRR042304.txt 3599 868
SRR8209869.txt 3166 806
SRR8209875.txt 1722 397
SRR8209841.txt 8776 4160
CRR055537.txt 606 77
CRR042296.txt 1147 159
SRR8209853.txt 1531 263
SRR8209845.txt 7557 3619
CRR055561.txt 504 43
SRR8209849.txt 1962 383
SRR8209857.txt 1245 184
SRR8209883.txt 1958 271
CRR055531.txt 1167 139
CRR055527.txt 2606 513
CRR055536.txt 697 79
CRR055555.txt 364 26
heart_IP_2.txt 3098 800
lung_IP_4.txt 3783 1196
stomach_IP_5.txt 1936 465
heart_IP_1.txt 1817 423
muscle_IP_4.txt 2962 768
liver_IP_3.txt 2545 549
SRR8209837.txt 2796 690
SRR8209839.txt 11012 5763
CRR055529.txt 1912 303
heart_IP_3.txt 2845 793
brain_IP_2.txt 4143 1618
CRR042302.txt 585 85
CRR055543.txt 772 75
CRR055539.txt 352 41
CRR055525.txt 3326 809
muscle_IP_5.txt 4358 1317
brain_IP_3.txt 4784 1808
SRR8209859.txt 889 96
CRR042308.txt 780 108
CRR073016.txt 526 69
CRR042282.txt 1301 242
SRR8209867.txt 1068 206
CRR055545.txt 961 125


In [2]:
# #### difference > 30%
### 如果Fisher exact test最终不能用，就用这种方法；

################################################################
data_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/sig/contained_m6A/highFC/rename/"
os.chdir(data_dir)
result_dir = "diff30%/"
os.system("mkdir -p %s" % result_dir)
##################################################################

def calculate_diff(in_txt):
    df = pd.read_table(in_txt, header=None)
    df.columns = ['contig', 'start', 'position', 'term', 'refRPKM_ratio', 'altRPKM_ratio', 'mark']
    df['mini'] = np.where(df['refRPKM_ratio']<df['altRPKM_ratio'], df['refRPKM_ratio'], df['altRPKM_ratio'])
    df['diff'] = np.abs(df['refRPKM_ratio'] - df['altRPKM_ratio'])
    df_sub = df[df['diff'] > (df['mini']*0.3)]
    result_file = os.path.join(result_dir, os.path.basename(in_txt))
    df_sub.to_csv(result_file, sep="\t", header=False, index=False)
#     print(in_txt, len(df_sub))

txt_list = glob.glob("*.bed")
for txt in txt_list:
    calculate_diff(txt)

#### Renlab allele m6A analysis
###### https://github.com/Jakob666/allele-specificM6A

##### pre-process

In [8]:
#### replace refseq accession to ensembl ID

def overlap_anno(bed):
    result_dir = "/Charles/project/ASm6A/peak_calling/MeTPeak/forRen_analysis/"
    gene = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/gencode.v19.annotation_Gene.bed"
    res_1 = os.path.join(result_dir, "%s.bed" % bed.split("/")[-2])
    os.system("bedtools intersect -a %s -b %s -wo > %s" % (bed, gene, res_1))
    df = pd.read_table(res_1, header=None)
    df.iloc[:,19] = df.iloc[:,19].str.split(".").str[0]
    df = df.iloc[:, [0,1,2,19,4,5,6,7,8,9,10,11]]
    res = res_1
    df.to_csv(res, sep="\t", header=False, index=False)
    
bed_dir = "/Charles/project/ASm6A/peak_calling/MeTPeak/"
bed_list = glob.glob("%s/*/peak.bed"%bed_dir)
for bed in bed_list:
#     print(bed)
    overlap_anno(bed)

##### Generate run shell

In [12]:
#######################################################################################
vcf_dir = "/Charles/project/ASm6A/by_ind/SNP_calling/filtered_vcf/forRen_analysis/"
peak_dir = "/Charles/project/ASm6A/peak_calling/MeTPeak/forRen_analysis/"
gtf = "/home/mjy/software/allele-specificM6A/Homo_sapiens.GRCh37.87.gtf"
software_dir = "/home/software/allele-specificM6A/allele-specificM6A-master/"
result_dir = "/Charles/project/ASm6A/ASm6A_renlab/"

ind_dict = {
    "ind1": ['brain_IP_1', 'heart_IP_1', 'liver_IP_1'],
    "ind2":
    ['brain_IP_2', 'heart_IP_2', 'placenta_IP_2', 'kidney_IP_2', 'liver_IP_2'],
    "ind3": ['brain_IP_3', 'heart_IP_3', 'liver_IP_3', 'kidney_IP_3'],
    "ind4": [
        'stomach_IP_4', 'muscle_IP_4', 'lung_IP_4', 'kidney_IP_4',
        'placenta_IP_4'
    ],
    "ind5": ['stomach_IP_5', 'muscle_IP_5', 'lung_IP_5'],
    "ind6": ['placenta_IP_6'],
    "ind7": [
        'Adipose-1-1', 'Adrenal_gland-1-1', 'Aorta-1-1', 'Heart-1-1',
        'Skin-1-1', 'Spleen-1-1'
    ],
    "ind8": [
        'Lung-2-1', 'Lung-2-4', 'Spleen-2-1', 'Tongue-2-1',
        'Urinary_bladder-2-1'
    ],
    "ind9":
    ['Appendix-3-2', 'Colon-3-2', 'Esophagus-3-2', 'Muscle-3-2', 'Spleen-3-2'],
    "ind10": [
        'Aorta-4-2', 'Esophagus-4-2', 'Heart-4-2', 'Jejunum-4-2', 'Liver-4-2',
        'Lung-4-2', 'Lung-4-4', 'Prostate-4-2', 'Rectum-4-2', 'Skin-4-2',
        'Stomach-4-2', 'Testis-4-2', 'Thyroid_gland-4-2', 'Urinary_bladder-4-2'
    ],
    "ind11": [
        'Appendix-5-3', 'Brainstem-5-3', 'Cerebellum-5-3', 'Cerebrum-5-3',
        'Duodenum-5-3', 'Hypothalamus-5-3', 'Jejunum-5-3', 'Muscle-5-3',
        'Rectum-5-3', 'Stomach-5-3', 'Thyroid_gland-5-3', 'Trachea-5-3',
        'Urinary_bladder-5-3'
    ],
    "ind12": ['Cerebrum-6-3'],
    "ind13": ['Cerebellum-7-4'],
    "ind14": [
        'FrontalCortex_1', 'Cerebellum_1', 'Heart_1', 'Liver_1', 'Lung_1',
        'Kidney_1', 'Spleen_1'
    ],
    "ind15": [
        'FrontalCortex_2', 'Cerebellum_2', 'Heart_2', 'Liver_2', 'Kidney_2',
        'Spleen_2', 'Muscle_1'
    ],
    "ind16": [
        'FrontalCortex_3', 'Cerebellum_3', 'Heart_3', 'Liver_3', 'Lung_2',
        'Muscle_2'
    ],
    "ind17": ['Muscle_3'],
    "ind18": ['Lung_3', 'Kidney_3', 'Spleen_3']
}

with open("%s/run.sh" % result_dir, 'w') as fw:
    for ind, sample_list in ind_dict.items():
        vcf = os.path.join(vcf_dir, "%s_gatk_het_SNP.vcf" % ind)
        for sample in sample_list:
            bed = os.path.join(peak_dir, "%s.bed" % sample)
            sub_dir = os.path.join(result_dir, sample)
            os.system("mkdir -p %s" % sub_dir)
            res = os.path.join(sub_dir, "%s.txt" % sample)
            commond_1 = "java -jar %s/renlabm6a_allele.jar AsmPeakDetection -g %s -bed %s -vcf %s -o %s -t 5" % (
                software_dir, gtf, bed, vcf, res)
            fw.write(commond_1 + "\n")
#         os.system(commond_1)

##### parse results

In [23]:
def pickOut_ASm6A(in_file):
    #     in_file = "/Charles/project/ASm6A/ASm6A_renlab/liver_IP_3/liver_IP_3.txt"
    df = pd.read_table(in_file)
    df = df[df['q-value'] < 0.05]
    asm6a_list = []
    for i, values in df.iterrows():
        chrom = values['#chr']
        peak_start = values['peakStart']
        peak_end = values['peakEnd']
        term_list = values['majorAlleleNC'].split(";")
        for term in term_list:
            pos, majorAllele = int(term.split(":")[0]), term.split(":")[1]
            start = pos - 1
            new_line = "%s\t%s\t%s\t%s\t%s\t%s\t%s" % (
                chrom, str(start), str(pos), majorAllele, chrom,
                str(peak_start), str(peak_end))
            asm6a_list.append(new_line)
    res = in_file.replace(".txt", ".bed")
    with open(res, 'w') as fw:
        fw.writelines(["%s\n" % x for x in asm6a_list])


data_dir = "/Charles/project/ASm6A/ASm6A_renlab/"  # liver_IP_3/liver_IP_3.txt
name_list = [
    os.path.basename(x).replace("/", "") for x in glob.glob("%s/*" % data_dir)
    if os.path.isdir(x)
]
sample_list = ["%s/%s/%s.txt" % (data_dir, name, name) for name in name_list]
# print(sample_list)
for sample in sample_list:
    pickOut_ASm6A(sample)

#### compare with data-derived methods

#### 这一部分的分析重新做，他们的方法是以peak为单位进行的，所以最后鉴定到的是ASm6A peaks.
#### 对于一个 m6A 信号峰，我们假设整个峰值区域存在一个全局的等位基因特异性优势比，
#### 该值可理解为修饰峰区域内所有 SNP 位点的等位基因特异性对数优势比的总体平均数(Population average)，
#### 则我们可以利用该值估计 m6A 信号峰的等位基因特异性大小。

In [3]:
# asm6a_list = glob.glob("/Charles/project/ASm6A/ASm6A_renlab/*/*.bed")
# result_dir = "/Charles/project/ASm6A/ASm6A_renlab/z_reformat/"

# for asm6a in asm6a_list:
#     df = pd.read_table(asm6a, header=None)
#     df.iloc[:, 0] = "chr" + df.iloc[:, 0].astype(str)
#     df.to_csv(os.path.join(result_dir, os.path.basename(asm6a)),
#               sep="\t",
#               header=False,
#               index=False)

In [5]:
#### restrict to same peak regions
# asm6a_list = glob.glob("/Charles/project/ASm6A/ASm6A_renlab/z_reformat/*.bed")
# peak_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_MSPC/rm_m6Am/rm_title/rename/"
# result_dir = "/Charles/project/ASm6A/ASm6A_renlab/z_comparision/restrict_to_samePeak/"

# for asm6a in asm6a_list:
#     peak = os.path.join(peak_dir, os.path.basename(asm6a))
#     res = os.path.join(result_dir, os.path.basename(asm6a))
#     os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" %
#               (asm6a, peak, res))

In [1]:
#### overlap with data-derived ASm6A results
# renlab_dir = "/Charles/project/ASm6A/ASm6A_renlab/z_comparision/restrict_to_samePeak/"
# result_dir = "/Charles/project/ASm6A/ASm6A_renlab/z_comparision/compare_results/"

# # asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
# asm6a_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/by_sample/"
# sample_list = glob.glob("%s/*.bed" % renlab_dir)
# for sample in sample_list:
#     asm6a = os.path.join(asm6a_dir, os.path.basename(sample))
#     res = os.path.join(result_dir, os.path.basename(sample))
#     os.system("bedtools intersect -a %s -b %s -wa | sort -u > %s" %
#               (asm6a, sample, res))
#     overlap_num = os.popen("wc -l %s" % res).read().split()[0]
#     asm6a_num = os.popen("wc -l %s" % asm6a).read().split()[0]
#     ren_num = os.popen("wc -l %s" % sample).read().split()[0]
#     print("%s\t%s:(%s)\t%s:(%s)" %
#           (os.path.basename(sample), ren_num,
#            round(int(overlap_num) * 100 / int(ren_num), 2), asm6a_num,
#            round(int(overlap_num) * 100 / int(asm6a_num), 2)))

#### reanalysis

In [29]:
# peak_dir = "/Charles/project/ASm6A/peak_calling/merged_peak_MSPC/rm_m6Am/rm_title/rename/"
result_dir = "/Charles/project/ASm6A/ASm6A_renlab/sig_peak/"
ren_dir = "/Charles/project/ASm6A/ASm6A_renlab/"
bed_list = glob.glob("%s/*/*.bed" % ren_dir)
for bed in bed_list:
    prefix = bed.split("/")[-2]
    df = pd.read_table(bed, sep="\t", header=None)
    df['chr'] = "chr" + df_ren.iloc[:,4].astype(str)
    df['start'] = df_ren.iloc[:,5].astype(str)
    df['end'] =  df_ren.iloc[:,6].astype(str)
    df = df[['chr','start','end']].drop_duplicates().dropna()
    res = os.path.join(result_dir, "%s.bed"%prefix)
    df.to_csv(res, sep="\t", index=False, header=False)

In [17]:
result_dir = "/Charles/project/ASm6A/ASm6A/sig_peak/"
asm6a_dir = "/Charles/project/ASm6A/ASm6A/withPeak/"
bed_list = glob.glob("%s/*.bed" % asm6a_dir)
for bed in bed_list:
    prefix = os.path.basename(bed).split(".bed")[0]
    df = pd.read_table(bed, sep="\t", header=None)
    df['chr'] = df.iloc[:,6]
    df['start'] = df.iloc[:,7].astype(str)
    df['end'] = df.iloc[:,8].astype(str)
    res = os.path.join(result_dir, "%s.bed"%prefix)
    df[['chr','start','end']].drop_duplicates().to_csv(res, sep="\t", index=False, header=False)

In [30]:
self_dir = "/Charles/project/ASm6A/ASm6A/sig_peak/"
ren_dir = "/Charles/project/ASm6A/ASm6A_renlab/sig_peak/"
bed_list = glob.glob("%s/*.bed" % self_dir)
for bed in bed_list:
    ren_bed = os.path.join(ren_dir, os.path.basename(bed))
    self_num = os.popen("wc -l %s" % bed).read().split()[0]
    overlap_num = os.popen("bedtools intersect -a %s -b %s -wa | sort -u | wc -l" % (bed, ren_bed)).read().split()[0]
#     print(self_num)
#     print(overlap_num)
    print(os.path.basename(bed), self_num, overlap_num, int(overlap_num)/ int(self_num))
#     break

Kidney_2.bed 36 1 0.027777777777777776
Liver_3.bed 135 16 0.11851851851851852
Spleen_3.bed 607 150 0.2471169686985173
stomach_IP_4.bed 708 234 0.3305084745762712
Appendix-3-2.bed 108 8 0.07407407407407407
Liver_1.bed 128 7 0.0546875
Cerebellum-5-3.bed 454 110 0.2422907488986784
Adrenal_gland-1-1.bed 699 113 0.16165951359084407
FrontalCortex_3.bed 409 96 0.23471882640586797
liver_IP_3.bed 620 128 0.2064516129032258
kidney_IP_4.bed 858 219 0.25524475524475526
Cerebellum_3.bed 509 124 0.24361493123772102
Aorta-4-2.bed 258 159 0.6162790697674418
Muscle_1.bed 70 14 0.2
Tongue-2-1.bed 141 20 0.14184397163120568
Cerebrum-6-3.bed 168 1 0.005952380952380952
Lung_2.bed 396 117 0.29545454545454547
Hypothalamus-5-3.bed 163 56 0.34355828220858897
Heart-4-2.bed 180 107 0.5944444444444444
Cerebrum-5-3.bed 252 71 0.28174603174603174
Spleen-3-2.bed 56 9 0.16071428571428573
Cerebellum_1.bed 311 30 0.09646302250803858
Heart-1-1.bed 640 133 0.2078125
Urinary_bladder-2-1.bed 48 5 0.10416666666666667
Lung-4